## Web Scraping 

In [53]:
import numpy as np
import pandas as pd
import requests
import datetime 

from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")

In [54]:
class Parser:

    def __init__(self, data=pd.DataFrame({'time': [], 'title': [], 'link': [], 'source': [], 'processed': []})):
        self.df = data
        self.interfax_url = 'https://www.interfax.ru/news/'
        self.lenta_url = 'https://lenta.ru/'

    def parse_interfax(self, date):
        
        res = pd.DataFrame({'time': [], 'title': [], 'link': [], 'source': [], 'processed': []})
        def get_right_link(x):
            if 'http' in x:
                return x
            else:
                return 'https://interfax.ru' + x

        for i in range(1, 20):
            response = requests.get(
                self.interfax_url + date + '/all/page_' + str(i))
            soup = BeautifulSoup(response.text, 'lxml')

            items = soup.find(class_='an')

            time_stamps = [date + item.text for item in items.find_all('span')]
            links = [get_right_link(item.get('href'))
                     for item in items.find_all('a')]
            titles = [item.find('h3').get_text().strip()
                      for item in items.find_all('a')]
            if links[0] in self.df['link'].values:
                break
                
            res = pd.concat([res, pd.DataFrame({'time': time_stamps, 
                                'title': titles, 'link': links, 'source': 'interfax'})])
        
        res['time'] = pd.to_datetime(res['time'], format='%Y/%m/%d%H:%M')
        
        
        return res

    def parse_lenta(self, date):
        
        res = pd.DataFrame({'time': [], 'title': [], 'link': [], 'source': [], 'processed': []})
        for i in range(1, 30):
            response = requests.get(self.lenta_url + date + '/page/' + str(i))
            soup = BeautifulSoup(response.text, 'lxml')

            items = soup.find_all(class_='archive-page__item _news')
            
            if len(items) == 0:
                break

            links = [self.lenta_url[:-1] +
                     item.find('a').get('href') for item in items]
            time_stamps = [date + item.find('time').text[:5] for item in items]
            titles = [item.find('h3').text for item in items]
            
            res = pd.concat([res, pd.DataFrame({'time': time_stamps, 
                                'title': titles, 'link': links, 'source': 'lenta'})])
        
        res['time'] = pd.to_datetime(res['time'], format='%Y/%m/%d%H:%M')
        
        return res

    def parse(self, n_days=30, mode='default'):

        if mode == 'update':
            start_date = self.df['time'].max()
            if start_date is np.nan:
                raise ValueError(
                    "Existing database is empty. Use mode='default' to fill it from scratch"
                )
            start_date, end_date = start_date.date(), datetime.date.today()
        else:
            start_date, end_date = datetime.date.today(
            ) - datetime.timedelta(days=n_days), datetime.date.today()
            
        delta = end_date - start_date
        dates = [str(start_date + datetime.timedelta(days=i))[:4] + '/'
                 + str(start_date + datetime.timedelta(days=i))[5:7] + '/'
                 + str(start_date + datetime.timedelta(days=i))[8:10] for i in range(delta.days + 1)]
        
        for date in tqdm(dates):
            self.df = pd.concat([self.df, self.parse_interfax(date)])
            self.df = pd.concat([self.df, self.parse_lenta(date)])
            
        self.df.drop_duplicates(subset='link', inplace=True)
        self.df.sort_values(by='time', inplace=True)
        self.df.reset_index(drop=True, inplace=True)

In [55]:
database = pd.read_pickle('df_24.12.22:58.pkl')

parser = Parser(data=database)
parser.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80475 entries, 0 to 80474
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       80475 non-null  datetime64[ns]
 1   title      80475 non-null  object        
 2   link       80475 non-null  object        
 3   source     80475 non-null  object        
 4   processed  80475 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 3.1+ MB


In [56]:
parser.parse(mode='update')
parser.df.info()

  0%|          | 0/2 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80637 entries, 0 to 80636
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       80637 non-null  datetime64[ns]
 1   title      80637 non-null  object        
 2   link       80637 non-null  object        
 3   source     80637 non-null  object        
 4   processed  80475 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 3.1+ MB


## Text Vectorizer

In [77]:
from pymystem3 import Mystem
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

tokenizer = TweetTokenizer()
m = Mystem()

def preprocess(text):

    def tokeniz(text): return ' '.join(tokenizer.tokenize(text.lower()))

    stop_words = set(stopwords.words('russian'))

    def delete_stop_words(text):

        filtered_tokens = []
        for token in text.split():
            if token not in stop_words:
                filtered_tokens.append(token)

        return ' '.join(filtered_tokens)

    def lemmatize_sentence(text):
        lemmas = m.lemmatize(text)
        return "".join(lemmas).strip()

    def symbol_deleting(text):
        alphabet = 'abcdefghijklmnopqrstuvwxyzабвгдежзийклмнопрстуфхцчшщъыьэюя0123456789 '

        text = text.replace("ё", "е")
        words = ''.join([[" ", i][i in alphabet] for i in text]).split()
        return ' '.join(words)

    text = tokeniz(text)
    text = delete_stop_words(text)
    text = symbol_deleting(text)
    text = lemmatize_sentence(text)

    return text

def fill_na(row):

    if isinstance(row['processed'], float):
        return preprocess(row['title'])
    else:
        return row['processed']
    

In [6]:
parser.df.apply(fill_na, axis=1, inplace=True)
parser.df.tail(10)

,time,title,link,source,processed
80621,2022-12-25 14:27:00,Турция и Россия обсудили открытие воздушного п...,https://lenta.ru/news/2022/12/25/turkish_rus/,lenta,турция россия обсуждать открытие воздушный про...
80622,2022-12-25 14:28:00,В провинции Чжэцзян в Китае выявляют за сутки ...,https://interfax.ru/world/878575,interfax,провинция чжэцзян китай выявлять сутки миллион...
80623,2022-12-25 14:31:00,«Человек-слон» рассказал о своем заболевании,https://lenta.ru/news/2022/12/25/elephant_man/,lenta,человек слон рассказывать свой заболевание
80624,2022-12-25 14:32:00,57-летняя актриса снялась в откровенном наряде...,https://lenta.ru/news/2022/12/25/eliza/,lenta,57 летний актриса сниматься откровенный наряд ...
80625,2022-12-25 14:38:00,Переехавшая в Аргентину россиянка описала стра...,https://lenta.ru/news/2022/12/25/argentina/,lenta,переезжать аргентина россиянка описывать стран...
80626,2022-12-25 14:41:00,На востоке и в центре Украины снова объявили в...,https://lenta.ru/news/2022/12/25/snova_ona/,lenta,восток центр украина снова объявлять воздушный...
80627,2022-12-25 14:46:00,Шойгу заявил о противодействии сил Запада росс...,https://lenta.ru/news/2022/12/25/shoygu_zapad/,lenta,шойгу заявлять противодействие сила запад росс...
80628,2022-12-25 14:48:00,Туктамышева сравнила призовые на чемпионатах Р...,https://lenta.ru/news/2022/12/25/tuktik/,lenta,туктамышева сравнивать призовой чемпионат росс...
80629,2022-12-25 14:49:00,Китай провел военные учения со стрельбами и па...,https://lenta.ru/news/2022/12/25/china_uchenie/,lenta,китай проводить военный учение стрельба патрул...
80630,2022-12-25 14:54:00,Большинство немцев выступили против поставок б...,https://lenta.ru/news/2022/12/25/tanki_protiv/,lenta,большинство немец выступать против поставка бо...


## People and Organizations

In [30]:
data = parser.df.drop(parser.df.loc[parser.df['time'] < pd.to_datetime(
    start_date - datetime.timedelta(days=28))].index).copy().reset_index()

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

# last 7 days
start_date = datetime.date.today() - datetime.timedelta(days=7)

new_indexes = data.loc[data['time'] >= pd.to_datetime(start_date)].index
old_indexes = data.loc[(data['time'] <= pd.to_datetime(start_date)) &
                    (data['time'] >= pd.to_datetime(start_date - datetime.timedelta(days=28)))].index

In [34]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

X = vectorizer.fit_transform(data['processed']).toarray()

vocab = vectorizer.get_feature_names_out()
vocab.shape

(96730,)

In [46]:
new_freq = np.asarray(X[new_indexes].sum(axis=0)).ravel() / new_indexes.shape[0]
old_freq = np.asarray(X[old_indexes].sum(axis=0)).ravel() / old_indexes.shape[0]
old_freq[old_freq == 0] = 1 / old_indexes.shape[0]

vocab = vectorizer.get_feature_names_out()
razn = np.array(new_freq / old_freq)

In [47]:
top = dict(zip(vocab, razn))
sorted(top.items(), key=lambda item: item[1], reverse=True)

[('кемерово', 123.24273221460648),
 ('престарелый', 123.24273221460648),
 ('дом престарелый', 107.83739068778065),
 ('зеленский сша', 77.02670763412905),
 ('визит зеленский', 65.4727014890097),
 ('престарелый кемерово', 65.4727014890097),
 ('приют', 53.91869534389033),
 ('пожар дом', 46.21602458047742),
 ('гостиница донецк', 42.364689198770975),
 ('месторождение иркутский', 42.364689198770975),
 ('белгород', 38.51335381706453),
 ('кемеровский', 38.51335381706453),
 ('обстрел гостиница', 38.51335381706453),
 ('газопровод чувашия', 34.66201843535807),
 ('поездка зеленский', 34.66201843535807),
 ('зеленский вашингтон', 30.81068305365162),
 ('рогозин', 30.81068305365162),
 ('на', 28.885015362798395),
 ('взрыв газопровод', 26.959347671945164),
 ('газовый месторождение', 26.959347671945164),
 ('египет турист', 26.959347671945164),
 ('закон наказание', 26.959347671945164),
 ('солист группа', 26.959347671945164),
 ('больница донецк', 23.10801229023871),
 ('встречать', 23.10801229023871),
 ('вт

In [50]:
mask = df['processed'].str.contains(r'гадот', na=True)
df[mask]

,time,title,link,source,processed
76956,2022-12-19 13:22:00,Из «Флэша» вырежут камео Супермена и Чудо-женщ...,https://lenta.ru/news/2022/12/19/flashscenes/,lenta,флеш вырезать камео супермен чудо женщина кави...
77535,2022-12-20 11:25:00,Джеймс Ганн прокомментировал слухи об увольнен...,https://lenta.ru/news/2022/12/20/gann_gal/,lenta,джеймс ганн прокомментировать слух увольнение ...
79827,2022-12-23 12:41:00,Стало известно о возвращении Галь Гадот к роли...,https://lenta.ru/news/2022/12/23/gal_fast/,lenta,становиться известно возвращение галь гадот ро...


In [51]:
top

{'000': 0.0,
 '02': 1.9256676908532262,
 '02 годовой': 3.8513353817064524,
 '02 млрд': 0.0,
 '02 старт': 0.0,
 '04': 0.0,
 '04 баррель': 0.0,
 '05': 0.0,
 '05 бо': 0.0,
 '06': 0.0,
 '06 общий': 0.0,
 '088': 0.0,
 '088 человек': 0.0,
 '09': 0.0,
 '09 баррель': 0.0,
 '096': 0.0,
 '096 человек': 0.0,
 '10': 0.8502948245325933,
 '10 10': 0.0,
 '10 12': 0.0,
 '10 18': 3.8513353817064524,
 '10 20': 0.0,
 '10 23': 0.0,
 '10 26': 0.0,
 '10 75': 0.0,
 '10 iv': 0.0,
 '10 агротуризм': 0.0,
 '10 балл': 0.0,
 '10 год': 0.0,
 '10 гонка': 0.0,
 '10 градус': 0.0,
 '10 декабрь': 0.0,
 '10 день': 0.0,
 '10 замедление': 0.0,
 '10 килограмм': 3.8513353817064524,
 '10 компания': 3.8513353817064524,
 '10 летие': 0.0,
 '10 летний': 0.9628338454266131,
 '10 месяц': 0.6418892302844087,
 '10 миллиард': 0.0,
 '10 миллион': 3.8513353817064524,
 '10 минута': 3.8513353817064524,
 '10 млн': 0.0,
 '10 млрд': 1.4442507681399195,
 '10 повышать': 3.8513353817064524,
 '10 процедура': 0.0,
 '10 процент': 3.851335381706452

In [52]:
sorted(top.items(), key=lambda item: item[1], reverse=True)

[('кемерово', 123.24273221460648),
 ('престарелый', 123.24273221460648),
 ('дом престарелый', 107.83739068778065),
 ('зеленский сша', 77.02670763412905),
 ('визит зеленский', 65.4727014890097),
 ('престарелый кемерово', 65.4727014890097),
 ('приют', 53.91869534389033),
 ('пожар дом', 46.21602458047742),
 ('гостиница донецк', 42.364689198770975),
 ('месторождение иркутский', 42.364689198770975),
 ('белгород', 38.51335381706453),
 ('кемеровский', 38.51335381706453),
 ('обстрел гостиница', 38.51335381706453),
 ('газопровод чувашия', 34.66201843535807),
 ('поездка зеленский', 34.66201843535807),
 ('зеленский вашингтон', 30.81068305365162),
 ('рогозин', 30.81068305365162),
 ('на', 28.885015362798395),
 ('взрыв газопровод', 26.959347671945164),
 ('газовый месторождение', 26.959347671945164),
 ('египет турист', 26.959347671945164),
 ('закон наказание', 26.959347671945164),
 ('солист группа', 26.959347671945164),
 ('больница донецк', 23.10801229023871),
 ('встречать', 23.10801229023871),
 ('вт

In [53]:
count

4214

In [55]:
X_all = np.asarray(X.sum(axis=0)).ravel()

In [56]:
vocab[np.argsort(X_all)[-100:]]

array(['переговоры', 'показывать', 'день', 'млрд', 'отвечать',
       'запорожский', 'россиянка', 'граница', 'бывший', 'один', 'польша',
       'запад', 'ракета', 'причина', 'президент', 'северный', 'летний',
       'город', 'байден', 'первый', 'ребенок', 'находить', 'днр', 'пожар',
       'госдума', 'мужчина', 'высказываться', 'мид', 'женщина',
       'отказываться', 'рост', 'удар', 'помощь', 'работа', 'дело', 'нато',
       'число', 'поставка', 'обвинять', 'самый', 'взрыв', 'обстрел',
       'германия', 'объяснять', 'свой', 'турция', 'мобилизация', 'кремль',
       'попадать', 'зеленский', 'предупреждать', 'известно', 'видео',
       'дом', 'суд', 'китай', 'начинать', 'получать', 'киев', 'санкция',
       'власть', 'против', 'мир', 'нефть', 'ес', 'способ', 'человек',
       'европа', 'минобороны', 'предлагать', 'регион', 'рубль',
       'призывать', 'цена', 'газ', 'украинский', 'глава', 'раскрывать',
       'оценивать', 'страна', 'военный', 'область', 'сообщать', 'москва',
       'вс

## Hackathon Trend Extractor

In [11]:
get_good_trends(get_all_trends(df, CountVectorizer()))

array(['ребенок', 'учение', 'инспекция', 'днр', 'литва', 'отдых',
       'стоимость', 'июль', 'территория', 'россиянка', 'виза', 'въезд',
       'фбр', 'европа', 'петербург', 'рубль', 'новофедоровка', 'призыв',
       'срок', 'россиянин', 'тайвань', 'трамп', 'обыск', 'аэродром',
       'транзит', 'аэс', 'китай', 'август', 'взрыв', 'крым'], dtype='<U13')

In [10]:
from natasha import Segmenter, NewsMorphTagger, Doc, NewsEmbedding
from sklearn.feature_extraction.text import CountVectorizer

def get_all_trends(corpus, vectorizer):
    X = vectorizer.fit_transform(corpus['processed']).toarray()

    count = 2000

    new_freq = np.asarray(X[:count].sum(axis=0)).ravel() / len(X[:count])
    old_freq = np.asarray(X[count:].sum(axis=0)).ravel() / len(X[count:])

    razn = np.array(new_freq - old_freq)
    vocab = np.array(vectorizer.get_feature_names_out())

    return vocab[np.argsort(razn)[-40:]]


def get_good_trends(result):
    segmenter = Segmenter()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)

    trend = []

    for gram in result:

        doc = Doc(gram)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        synt_length = len(doc.tokens)

        if synt_length == 1 and doc.tokens[0].pos == 'NOUN':
            trend.append(gram)

        elif synt_length > 1:
            trend.append(gram)

    return np.array(trend)